# Reihenfolge auf Maschinen

In [1]:
from configs.path_manager import get_path
import src.utils.analize.sequence_tester as seq_tester

import pandas as pd
pd.set_option('display.max_rows', 26)
pd.set_option('display.max_colwidth', 93)

In [2]:
# pip install Levenshtein

In [3]:
data_path = get_path("data", "cp_lateness")

def get_file_path(prefix: str = "", text: str = "schedule"):
    return data_path / f"{prefix}_{text}.csv"

In [4]:
day_numb = 1

get_file_path(prefix="02", text=f"schedule_{day_numb:02d}_init")

WindowsPath('C:/Users/konra/Coding/HTWD_Minimalinvasives_Job-Shop_Scheduling/data/cp_lateness/02_schedule_01_init.csv')

In [5]:
day_numb = 1

df_plan_init =  pd.read_csv(get_file_path(prefix="02", text=f"schedule_{day_numb:02d}_init"))
df_plan_init = df_plan_init.sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_plan_init

,Job,Production_Plan_ID,Operation,Machine,Arrival,Deadline,Start,Processing Time,End,Lateness,Tardiness,Earliness
0,0,0,0,M00,49,2027,1440,29,1469,-558,0,558
1,0,0,1,M01,49,2027,1486,78,1564,-463,0,463
2,0,0,2,M02,49,2027,1628,9,1637,-390,0,390
3,0,0,3,M03,49,2027,1637,36,1673,-354,0,354
4,0,0,4,M04,49,2027,1703,49,1752,-275,0,275
...,...,...,...,...,...,...,...,...,...,...,...,...
195,19,6,5,M05,1398,3481,2483,21,2504,-977,0,977
196,19,6,6,M09,1398,3481,2504,32,2536,-945,0,945
197,19,6,7,M08,1398,3481,2683,89,2772,-709,0,709
198,19,6,8,M07,1398,3481,2772,30,2802,-679,0,679


## Mit "Deviation Penalty"

### Tag 2

In [6]:
day_numb = 2
file = get_file_path(prefix="04_devpen", text=f"schedule_{day_numb:02d}")

df_plan_temp = pd.read_csv(file)
df_plan_temp = df_plan_temp.sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_plan_temp 

df_plan_devpen_01 = df_plan_temp 
df_plan_devpen_01

,Job,Production_Plan_ID,Operation,Machine,Arrival,Deadline,Start,Processing Time,End,Lateness,Tardiness,Earliness
0,2,2,9,M04,151,2994,2938,33,2971,-23,0,23
1,3,3,8,M09,226,3505,2901,22,2923,-582,0,582
2,3,3,9,M05,226,3505,3462,43,3505,0,0,0
3,5,5,9,M07,464,2947,2918,25,2943,-4,0,4
4,7,7,9,M03,506,3204,3125,79,3204,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
239,40,4,5,M04,2862,4829,4044,69,4113,-716,0,716
240,40,4,6,M08,2862,4829,4165,21,4186,-643,0,643
241,40,4,7,M07,2862,4829,4263,49,4312,-517,0,517
242,40,4,8,M09,2862,4829,4312,72,4384,-445,0,445


#### Vergleich (Kendall Tau & Levenshtein)

In [7]:
df_compare_01 = seq_tester.analyze_machine_sequences(df_plan_init, df_plan_devpen_01, 1440*day_numb)
df_compare_01

,original_sequence,revised_sequence_raw,revised_sequence,changed,kendall_tau,levenshtein_distance
Machine,,,,,,
M03,"[7, 17, 15]","[24, 7, 17, 27, 23, 22, 33, 25, 28, 40, 39, 15, 29, 30, 20, 38, 21, 26, 37, 32, 34, 31, 3...","[7, 17, 15]",False,1.000000,0
M04,"[12, 18, 2, 19, 13]","[2, 18, 11, 12, 20, 19, 27, 21, 28, 29, 13, 25, 39, 35, 24, 40, 36, 38, 23, 22, 26, 31, 3...","[2, 18, 12, 19, 13]",True,0.400000,2
M05,"[3, 17]","[13, 24, 27, 20, 3, 23, 40, 25, 22, 33, 28, 35, 32, 38, 39, 30, 29, 26, 34, 17, 21, 31, 3...","[3, 17]",False,1.000000,0
M06,"[13, 14]","[17, 18, 13, 14, 20, 25, 33, 21, 23, 22, 28, 26, 34, 24, 27, 29, 39, 36, 30, 35, 38, 31, ...","[13, 14]",False,1.000000,0
M07,"[13, 5, 17, 15, 9, 11, 18]","[13, 5, 12, 15, 17, 9, 11, 22, 27, 25, 18, 20, 28, 24, 21, 23, 40, 39, 38, 33, 26, 29, 35...","[13, 5, 15, 17, 9, 11, 18]",True,0.904762,2
M08,"[17, 8, 16, 12]","[15, 17, 24, 8, 20, 22, 27, 25, 16, 21, 12, 26, 28, 34, 40, 36, 33, 38, 35, 23, 39, 32, 2...","[17, 8, 16, 12]",False,1.000000,0
M09,"[15, 13, 17]","[10, 3, 15, 13, 24, 17, 20, 25, 23, 33, 29, 22, 27, 26, 28, 34, 30, 40, 21, 38, 31, 35, 3...","[15, 13, 17]",False,1.000000,0


### Tag 3

In [8]:
day_numb = 3
file = get_file_path(prefix="04_devpen", text=f"schedule_{day_numb:02d}")

df_plan_temp = pd.read_csv(file)
df_plan_temp = df_plan_temp .sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_plan_temp 

df_plan_devpen_02 = df_plan_temp 
df_plan_devpen_02

,Job,Production_Plan_ID,Operation,Machine,Arrival,Deadline,Start,Processing Time,End,Lateness,Tardiness,Earliness
0,17,3,9,M05,1285,4564,4521,43,4564,0,0,0
1,21,3,8,M09,1495,4774,4416,22,4438,-336,0,336
2,21,3,9,M05,1495,4774,4731,43,4774,0,0,0
3,22,2,9,M04,1521,4364,4331,33,4364,0,0,0
4,23,8,8,M04,1557,4546,4419,26,4445,-101,0,101
...,...,...,...,...,...,...,...,...,...,...,...,...
232,57,7,5,M06,4281,6979,5503,88,5591,-1388,0,1388
233,57,7,6,M08,4281,6979,5591,19,5610,-1369,0,1369
234,57,7,7,M09,4281,6979,5649,48,5697,-1282,0,1282
235,57,7,8,M07,4281,6979,5938,36,5974,-1005,0,1005


In [9]:
og = df_plan_devpen_01
new = df_plan_devpen_02

df_compare_02 = seq_tester.analyze_machine_sequences(og, new, 1440*day_numb)
df_compare_02

,original_sequence,revised_sequence_raw,revised_sequence,changed,kendall_tau,levenshtein_distance
Machine,,,,,,
M00,[32],"[42, 47, 56, 44, 50, 55, 48, 46, 32, 49, 43, 54, 57, 53, 41, 45, 51, 52]",[32],False,NaN,0
M01,[31],"[56, 44, 42, 43, 41, 31, 49, 51, 45, 53, 55, 48, 46, 57, 54, 52, 47, 50]",[31],False,NaN,0
M02,[37],"[46, 57, 37, 41, 42, 44, 47, 50, 56, 45, 51, 49, 55, 53, 43, 52, 54, 48]",[37],False,NaN,0
M03,"[37, 32, 34, 31, 36, 35]","[37, 26, 32, 34, 42, 31, 56, 36, 41, 43, 49, 44, 35, 47, 50, 52, 55, 54, 48, 53, 46, 45, ...","[37, 32, 34, 31, 36, 35]",False,1.000000,0
M04,"[22, 26, 31, 33, 34, 32, 30, 37]","[22, 25, 23, 26, 31, 33, 34, 42, 47, 32, 30, 50, 44, 46, 41, 57, 37, 49, 55, 45, 51, 52, ...","[22, 26, 31, 33, 34, 32, 30, 37]",False,1.000000,0
M05,"[17, 21, 31, 37, 36]","[34, 17, 41, 21, 31, 42, 37, 44, 49, 56, 46, 57, 52, 53, 43, 50, 47, 54, 55, 48, 36, 45, 51]","[17, 21, 31, 37, 36]",False,1.000000,0
M06,"[38, 31, 40, 32, 37]","[30, 35, 38, 44, 31, 40, 32, 42, 37, 49, 53, 41, 46, 47, 50, 57, 43, 55, 45, 52, 54, 48, ...","[38, 31, 40, 32, 37]",False,1.000000,0
M07,"[38, 33, 26, 29, 35, 36, 31, 37, 32, 30, 34]","[40, 39, 38, 33, 26, 29, 35, 36, 31, 37, 32, 42, 30, 34, 56, 49, 43, 41, 46, 47, 55, 45, ...","[38, 33, 26, 29, 35, 36, 31, 37, 32, 30, 34]",False,1.000000,0
M08,"[33, 38, 35, 23, 39, 32, 29, 37, 31, 30]","[33, 38, 35, 23, 39, 36, 32, 44, 29, 37, 31, 42, 41, 56, 30, 53, 49, 43, 46, 52, 57, 45, ...","[33, 38, 35, 23, 39, 32, 29, 37, 31, 30]",False,1.000000,0


## Ohne "Deviation Penalty"

### Tag 2

In [10]:
day_numb = 2
file = get_file_path(prefix="03_simple", text=f"schedule_{day_numb:02d}")

df_plan_temp = pd.read_csv(file)
df_plan_temp = df_plan_temp .sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_plan_temp 

df_plan_simple_01 = df_plan_temp 
df_plan_simple_01

,Job,Production_Plan_ID,Operation,Machine,Arrival,Deadline,Start,Processing Time,End,Lateness,Tardiness,Earliness
0,2,2,9,M04,151,2994,2961,33,2994,0,0,0
1,3,3,8,M09,226,3505,2901,22,2923,-582,0,582
2,3,3,9,M05,226,3505,3462,43,3505,0,0,0
3,5,5,9,M07,464,2947,2922,25,2947,0,0,0
4,7,7,9,M03,506,3204,3125,79,3204,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
239,40,4,5,M04,2862,4829,4045,69,4114,-715,0,715
240,40,4,6,M08,2862,4829,4309,21,4330,-499,0,499
241,40,4,7,M07,2862,4829,4585,49,4634,-195,0,195
242,40,4,8,M09,2862,4829,4663,72,4735,-94,0,94


In [11]:
og = df_plan_init
new = df_plan_simple_01

df_compare_simple_01 = seq_tester.analyze_machine_sequences(og, new, 1440*day_numb)
df_compare_simple_01

,original_sequence,revised_sequence_raw,revised_sequence,changed,kendall_tau,levenshtein_distance
Machine,,,,,,
M03,"[7, 17, 15]","[32, 7, 25, 38, 27, 22, 37, 21, 39, 28, 24, 15, 33, 23, 40, 31, 20, 17, 30, 26, 29, 34, 3...","[7, 15, 17]",True,0.333333,2
M04,"[12, 18, 2, 19, 13]","[2, 18, 11, 21, 20, 32, 38, 19, 31, 27, 39, 28, 12, 35, 13, 25, 36, 24, 40, 29, 22, 23, 2...","[2, 18, 19, 12, 13]",True,0.200000,3
M05,"[3, 17]","[13, 32, 38, 20, 27, 25, 3, 24, 37, 35, 39, 28, 40, 33, 23, 22, 31, 26, 34, 30, 17, 29, 2...","[3, 17]",False,1.000000,0
M06,"[13, 14]","[17, 18, 14, 13, 32, 21, 25, 20, 37, 26, 35, 28, 33, 24, 34, 27, 22, 31, 23, 39, 36, 29, ...","[14, 13]",True,-1.000000,2
M07,"[13, 5, 17, 15, 9, 11, 18]","[13, 5, 17, 15, 9, 12, 21, 38, 11, 37, 20, 25, 27, 18, 28, 24, 22, 33, 23, 39, 36, 35, 26...","[13, 5, 17, 15, 9, 11, 18]",False,1.000000,0
M08,"[17, 8, 16, 12]","[17, 15, 32, 21, 38, 8, 20, 37, 25, 27, 16, 24, 12, 22, 28, 33, 26, 35, 34, 40, 36, 39, 2...","[17, 8, 16, 12]",False,1.000000,0
M09,"[15, 13, 17]","[10, 3, 15, 32, 13, 25, 31, 21, 20, 38, 37, 24, 27, 33, 23, 28, 22, 26, 34, 17, 29, 35, 3...","[15, 13, 17]",False,1.000000,0


### Tag 3

In [12]:
day_numb = 3
file = get_file_path(prefix="03_simple", text=f"schedule_{day_numb:02d}")

df_plan_temp = pd.read_csv(file)
df_plan_temp = df_plan_temp .sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_plan_temp 

df_plan_simple_02 = df_plan_temp 
df_plan_simple_02

,Job,Production_Plan_ID,Operation,Machine,Arrival,Deadline,Start,Processing Time,End,Lateness,Tardiness,Earliness
0,17,3,8,M09,1285,4564,4455,22,4477,-87,0,87
1,17,3,9,M05,1285,4564,4521,43,4564,0,0,0
2,21,3,9,M05,1495,4774,4731,43,4774,0,0,0
3,22,2,9,M04,1521,4364,4327,33,4360,-4,0,4
4,23,8,6,M06,1557,4546,4360,40,4400,-146,0,146
...,...,...,...,...,...,...,...,...,...,...,...,...
230,57,7,5,M06,4281,6979,5437,88,5525,-1454,0,1454
231,57,7,6,M08,4281,6979,5609,19,5628,-1351,0,1351
232,57,7,7,M09,4281,6979,5628,48,5676,-1303,0,1303
233,57,7,8,M07,4281,6979,5688,36,5724,-1255,0,1255


In [13]:
og = df_plan_simple_01
new = df_plan_simple_02

df_compare_simple_02 = seq_tester.analyze_machine_sequences(og, new, 1440*day_numb)
df_compare_simple_02

,original_sequence,revised_sequence_raw,revised_sequence,changed,kendall_tau,levenshtein_distance
Machine,,,,,,
M01,[29],"[43, 41, 29, 44, 45, 46, 42, 55, 52, 53, 56, 49, 51, 54, 57, 48, 47, 50]",[29],False,NaN,0
M02,[30],"[41, 46, 52, 44, 45, 42, 57, 55, 30, 53, 47, 51, 49, 56, 43, 50, 48, 54]",[30],False,NaN,0
M03,"[26, 29, 34, 36, 35]","[29, 26, 41, 34, 30, 42, 43, 55, 52, 44, 56, 49, 35, 45, 47, 53, 36, 51, 50, 48, 54, 46, 57]","[29, 26, 34, 35, 36]",True,0.600000,4
M04,"[22, 23, 26, 33, 34, 30, 37]","[22, 24, 23, 26, 33, 46, 42, 55, 45, 34, 30, 44, 47, 51, 57, 52, 41, 37, 49, 53, 50, 43, ...","[22, 23, 26, 33, 34, 30, 37]",False,1.000000,0
M05,"[34, 30, 17, 29, 21, 36]","[26, 41, 17, 34, 29, 46, 21, 31, 42, 30, 52, 44, 55, 57, 49, 53, 56, 47, 43, 36, 48, 50, ...","[17, 34, 29, 21, 30, 36]",True,0.466667,4
M06,"[36, 29, 38, 30, 40]","[23, 27, 39, 29, 38, 44, 36, 31, 40, 42, 46, 30, 45, 53, 55, 41, 52, 49, 51, 57, 47, 56, ...","[29, 38, 36, 40, 30]",True,0.400000,3
M07,"[39, 36, 35, 26, 40, 29, 31, 30, 32, 34]","[24, 23, 39, 33, 26, 40, 29, 35, 31, 32, 42, 30, 55, 34, 45, 46, 36, 49, 41, 51, 56, 57, ...","[39, 26, 40, 29, 35, 31, 32, 30, 34, 36]",True,0.466667,6
M08,"[36, 39, 23, 29, 31, 30]","[40, 35, 34, 33, 23, 39, 41, 44, 29, 36, 31, 46, 42, 52, 45, 30, 55, 53, 49, 51, 56, 57, ...","[23, 39, 29, 36, 31, 30]",True,0.466667,3
M09,"[17, 29, 35, 30, 39, 36, 40]","[29, 26, 35, 17, 34, 39, 40, 41, 44, 30, 46, 42, 52, 47, 49, 53, 45, 57, 36, 50, 56, 51, ...","[29, 35, 17, 39, 40, 30, 36]",True,0.523810,5
